In [1]:
from typing import List

"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
import inspect
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

import minlora
# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 200
log_interval = 1
eval_iters = 20
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'united'
wandb_run_name = 'gpt2' # 'run' + str(time.time())
# data
dataset = 'openwebtext'
gradient_accumulation_steps = 5 # used to simulate larger batch sizes
batch_size = 3 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 20 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'mps' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float32' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = False # use PyTorch 2.0 to compile the model to be faster
use_lora = True
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
#exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    print('Will use ddp')
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
else:
    print('Will not use ddp')
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    gradient_accumulation_steps *= 8 # simulate 8 gpus

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Will not use ddp


In [2]:
data_dir = os.path.join('data', dataset)

In [3]:
from functools import partial

import torch
from minlora import LoRAParametrization

learning_rate = 1e-3 # use a higher LR for LoRA
lora_dropout_p = 0.0
rank=4
lora_alpha = 64
lora_config = {
    torch.nn.Embedding: {
        "weight": partial(LoRAParametrization.from_embedding, rank=rank, lora_alpha=lora_alpha),
    },
    torch.nn.Linear: {
        "weight": partial(LoRAParametrization.from_linear, rank=rank, lora_alpha=lora_alpha),
    },
}

In [4]:
cache_dir = '/Volumes/WD_RED_RW/machine_learning/cache'

import datasets
import os
from tqdm import tqdm
from datasets.builder import DatasetGenerationError

# Define the path to the folders containing the text files
base = '/Users/seven/Downloads/corpus/'
parent_folders = [
    "Arzamas",
    "Interfax",
    "Lenta",
    "NPlus1",
]
for year in range(2008, 2018):
    parent_folders.append(f"proza_ru_{year}" )

# Define a function to recursively search for text files
def find_text_files(folder_path):
    text_files = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.startswith('.'):
                continue
            if file_name.endswith('.txt'):
                file_path = os.path.join(root, file_name)
                text_files.append(file_path)
        for sub_dir in dirs:
            sub_dir_path = os.path.join(root, sub_dir)
            text_files += find_text_files(sub_dir_path)
    return text_files

# Define the function to read text files from a folder
def read_files_from_folder(folder_path):
    # Find all the text files in the given folder and its subdirectories
    text_files = find_text_files(folder_path)

    # Iterate through all the text files
    for file_path in text_files:
        # Read the text from the file
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
        # Yield the text
        yield {"text": text}

# Create the dataset using the read_files_from_folder function
datasets_dict = datasets.DatasetDict()
for parent_folder in parent_folders:
    try:
        print(f'Processing {parent_folder}')
        folder_path = os.path.join(base, parent_folder)
        dataset = datasets.Dataset.from_generator(
            generator=lambda: read_files_from_folder(folder_path),
            cache_dir=cache_dir
            #features=Features.from_dict({"text": datasets.Value("string")}),
        )
        datasets_dict[parent_folder] = dataset
    except DatasetGenerationError:
        print(f'Will skip {parent_folder}')
        continue


Processing Arzamas


Found cached dataset generator (/Volumes/WD_RED_RW/machine_learning/cache/generator/default-90a9f236839f8254/0.0.0)
Found cached dataset generator (/Volumes/WD_RED_RW/machine_learning/cache/generator/default-311e4af8de1955ef/0.0.0)


Processing Interfax


Found cached dataset generator (/Volumes/WD_RED_RW/machine_learning/cache/generator/default-dcdb81eadbeb90a7/0.0.0)


Processing Lenta


...

Processing proza_ru_2015


Found cached dataset generator (/Volumes/WD_RED_RW/machine_learning/cache/generator/default-d833c37421182561/0.0.0)


Processing proza_ru_2016


Found cached dataset generator (/Volumes/WD_RED_RW/machine_learning/cache/generator/default-377705c522217a49/0.0.0)


Processing proza_ru_2017


In [5]:
from datasets import load_dataset, interleave_datasets, Dataset, IterableDataset

datasets_ru: list[Dataset | IterableDataset] = []
for k, v in datasets_dict.items():
    v = v.to_iterable_dataset(num_shards=128)
    datasets_ru.append(v)
print(datasets_ru[0])

In [6]:
from datasets import load_dataset, interleave_datasets

# Load data
pile = load_dataset("EleutherAI/the_pile_deduplicated", split="train", streaming=True)
pile


In [7]:
all_datasets = datasets_ru
all_datasets.append(pile)
multi_dataset = interleave_datasets(all_datasets)

shuffled_dataset = multi_dataset.shuffle(buffer_size=10_000, seed=42)

train_dataset = shuffled_dataset.skip(1000)
validation_dataset = shuffled_dataset.take(1000)

In [8]:
import torch
import numpy as np
import tiktoken
import threading
import queue

enc = tiktoken.get_encoding("gpt2")
MAX_LENGTH = block_size

train_queue = queue.Queue(maxsize=500)
validation_queue = queue.Queue(maxsize=500)

# Define a function to download and preprocess the data in a background thread
def data_worker():
    #print(f'data_worker will download')
    global train_queue, validation_queue
    while True:
        # Check if there is space in the data queue
        if train_queue.qsize() < train_queue.maxsize:
            data = train_dataset.shuffle(buffer_size=1000)
            for d in data:
                if train_queue.qsize() == train_queue.maxsize:
                    break
                ids = enc.encode_ordinary(d['text'])
                ids.append(enc.eot_token)
                train_queue.put(ids)
        if validation_queue.qsize() < validation_queue.maxsize:
            data = validation_dataset.shuffle(buffer_size=1000)
            for d in data:
                if validation_queue.qsize() == validation_queue.maxsize:
                    break
                ids = enc.encode_ordinary(d['text'])
                ids.append(enc.eot_token)
                validation_queue.put(ids)
        else:
            # Wait for the batch queue to empty
            if train_queue.qsize() == train_queue.maxsize and validation_queue.qsize() == validation_queue.maxsize:
                break


In [9]:
def get_batch(split):
    #data = train_dataset if split == 'train' else validation_dataset
    buffer_ids = []
    #print('Will shuffle data')
    #data = data.shuffle(buffer_size=1000)

    result_ids = []
    i = 0
    while True:
        if len(result_ids) == batch_size:
            break
        if split == 'train':
            if train_queue.empty():
                data_worker()
            ids = train_queue.get()
        else:
            if validation_queue.empty():
                data_worker()
            ids = validation_queue.get()

        buffer_ids = buffer_ids + ids

        while len(buffer_ids) > MAX_LENGTH + 1:
            if len(result_ids) == batch_size:
                break
            # Truncate text if it's longer than MAX_LENGTH
            id = buffer_ids[:MAX_LENGTH + 1]
            result_ids.append(id)
            buffer_ids = buffer_ids[MAX_LENGTH:]

            # Pad text if it's shorter than MAX_LENGTH
        i = i + 1

    #print(f'train_queue size {train_queue.qsize()}, val: {validation_queue.qsize()}')
    #batch_text = [data[i]['text'] for i in range(batch_size)]
    x = torch.stack([torch.tensor(ids)[:-1] for ids in result_ids])
    y = torch.stack([torch.tensor(ids)[1:] for ids in result_ids])

    if device == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# Get a batch of training data
x, y = get_batch('train')
print(f'x shape: {x.shape}')  # should print "x shape: torch.Size([12, 1024])"
print(f'y shape: {y.shape}')  # should print "y shape: torch.Size([12, 1024])"

x shape: torch.Size([3, 1024])
y shape: torch.Size([3, 1024])


In [10]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)
# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
if use_lora:
    minlora.add_lora(model, lora_config=lora_config)
    minlora.tie_weights(linear=model.lm_head, embedding=model.transformer.wte)
model.to(device)

Initializing a new model from scratch
defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)
number of parameters: 123.59M


GPT(
  (transformer): ModuleDict(
    (wte): ParametrizedEmbedding(
      50304, 768
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): LoRAParametrization()
        )
      )
    )
    (wpe): ParametrizedEmbedding(
      1024, 768
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): LoRAParametrization()
        )
      )
    )
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): ParametrizedLinear(
            in_features=768, out_features=2304, bias=False
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): LoRAParametrization()
              )
            )
          )
          (c_proj): ParametrizedLinear(
            in_features=768, out_features=768, bias=False
            (parametrizations): ModuleDict(
              (weight): Para

In [11]:
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
def configure_optimizers_lora(self, weight_decay, learning_rate, betas, device_type):
    # we apply weight decay to all lora params
    optim_groups = [
        # note: .get_lora_params() returns a generator
        # we need to wrap it in a list so we can consume it twice
        {"params": list(minlora.get_lora_params(self)) , "weight_decay": weight_decay},
        # you can also add biases for fine-tuning,
        # but I want to make sure lora alone works
        # {"params": minlora.get_bias_params(self), "weight_decay": 0.0}, # bias params don't get weight decay
    ]

    def parameter_count(optim_groups):
        n = sum(p.numel() for group in optim_groups for p in group["params"])
        if n < 1e6:
            return f"{n/1e3:.1f}k"
        else:
            return f"{n/1e6:.1f}M"

    print(f"optimizing {parameter_count(optim_groups)} parameters")

    # new PyTorch nightly has a new 'fused' option for AdamW that is much faster
    use_fused = (device_type == "cuda") and ("fused" in inspect.signature(torch.optim.AdamW).parameters)
    print(f"using fused AdamW: {use_fused}")
    extra_args = dict(fused=True) if use_fused else dict()
    optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)

    return optimizer
if use_lora:
    optimizer = configure_optimizers_lora(model, weight_decay, learning_rate, (beta1, beta2), device_type)
else:
    optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)


optimizing 801.3k parameters
using fused AdamW: False


In [12]:
# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
running_mfu = -1.0

In [13]:
raw_model

GPT(
  (transformer): ModuleDict(
    (wte): ParametrizedEmbedding(
      50304, 768
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): LoRAParametrization()
        )
      )
    )
    (wpe): ParametrizedEmbedding(
      1024, 768
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): LoRAParametrization()
        )
      )
    )
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): ParametrizedLinear(
            in_features=768, out_features=2304, bias=False
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): LoRAParametrization()
              )
            )
          )
          (c_proj): ParametrizedLinear(
            in_features=768, out_features=768, bias=False
            (parametrizations): ModuleDict(
              (weight): Para

In [ ]:
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                if use_lora:
                    checkpoint['lora'] = minlora.get_lora_state_dict(raw_model)
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            # in DDP training we only need to sync gradients at the last micro step.
            # the official way to do this is with model.no_sync() context manager, but
            # I really dislike that this bloats the code and forces us to repeat code
            # looking at the source of that context manager, it just toggles this variable
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
        with ctx:
            logits, loss = model(X, Y)
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        # backward pass, with gradient scaling if training in fp16
        #print(f'Will scale and backward {micro_step} from {gradient_accumulation_steps}')
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        print('Will clip_grad_norm')
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    print('Will step')
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    print('Will update')
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if True or iter_num % log_interval == 0 and master_process:
        lossf = loss.item() # loss as float. note: this is a CPU-GPU sync point
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

if ddp:
    destroy_process_group()

step 0: train loss 10.8959, val loss 10.8899
Will clip_grad_norm
Will step
Will update
iter 0: loss 10.8782, time 54169.51ms, mfu -100.00%
Will clip_grad_norm
Will step
Will update
iter 1: loss 10.8979, time 42671.29ms, mfu -100.00%
Will clip_grad_norm
Will step
Will update
iter 2: loss 10.6887, time 42498.65ms, mfu -100.00%
Will clip_grad_norm
Will step
Will update
iter 3: loss 10.2646, time 42414.63ms, mfu -100.00%
Will clip_grad_norm
Will step
Will update
iter 4: loss 9.9771, time 42286.03ms, mfu -100.00%
Will clip_grad_norm
Will step
Will update
iter 5: loss 9.1891, time 58119.84ms, mfu 0.58%
Will clip_grad_norm
Will step
Will update
iter 6: loss 9.7284, time 42279.96ms, mfu 0.60%
Will clip_grad_norm
Will step
Will update
iter 7: loss 8.4189, time 42266.84ms, mfu 0.62%
Will clip_grad_norm
Will step
Will update
iter 8: loss 8.1264, time 41892.46ms, mfu 0.64%
Will clip_grad_norm
Will step
Will update
iter 9: loss 7.8557, time 41854.66ms, mfu 0.66%
Will clip_grad_norm
Will step
Will u